In [1]:
from tensorflow.keras.applications import MobileNetV3Small
#from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten  , Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop, Adam
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LeakyReLU


# constantes para arquitectura, generación datos y entrenamiento/validacion
IMG_SIZE = (96, 96, 3)  # tamaño de la imagen de entrada
N_CLASSES = 16          # cantidad de clases a identificar
LOTES = 64             # tamaño de lotes para entrenamiento y testeo 
ALFA = 0.00005           # tasa de entrenamiento
EPOCAS = 1000           # cantidad de epocas a entrenar como máximo
PACIENCIA = 25          # parámetro para parada temprana
SEP_TRAIN_TEST = 0.2    # división en entrenamiento y validación 

# %% construye le modelo
# obtiene la arquitectura para el modelo pre-entrenado
feature_model = MobileNetV3Small(
    #input_shape=IMG_SIZE,
    minimalistic=False,
    #include_top=False,
    weights="imagenet",
    include_preprocessing=True,
)

# construye el modelo de manera funcional
layers = Flatten() (feature_model.output)
layers = Dense(256, activation=LeakyReLU())(layers)
layers = BatchNormalization()(layers)
layers = Dropout(0.2)(layers)
output = Dense(N_CLASSES, activation='softmax')(layers)

model = Model(inputs=feature_model.input, outputs=output)

# The newly added layers are initialized with random values.
# Make sure based model remain unchanged until newly added layers weights get reasonable values.
for layer in feature_model.layers:
    layer.trainable = False

#Optimizer
opt = Adam(learning_rate=ALFA)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


# %% Entrenamiento del modelo 

IMAGES_DIR = '../Datos/flowers/'

# construye un generador de imagenes para dividir en entrenamiento y validación
data_generator = ImageDataGenerator(
    validation_split=SEP_TRAIN_TEST
)

# generador para entrenamiento a partir de la carpeta indicada en IMAGES_DIR
train_iter = data_generator.flow_from_directory(
            target_size=(IMG_SIZE[0],IMG_SIZE[1]),
            directory=IMAGES_DIR, 
            class_mode='categorical',
            batch_size=LOTES,
            subset='training'   # asigna subconjunto segun validation_split del ImageDataGenerator
            )

valid_iter = data_generator.flow_from_directory(
            target_size=(IMG_SIZE[0],IMG_SIZE[1]),
            directory=IMAGES_DIR, 
            class_mode='categorical',
            batch_size=LOTES,
            subset='validation' # asigna subconjunto segun validation_split del ImageDataGenerator
            )

# Callback para parada temprana
early_stop = EarlyStopping(monitor='val_loss', 
                           patience=PACIENCIA,  
                           restore_best_weights=True)

H = model.fit(
    train_iter,
    validation_data=valid_iter,
    validation_steps= 16,
    epochs=EPOCAS,
    callbacks=[early_stop])

# dibuja accuracy del progreso del entrenamiento
fig, axs = plt.subplots(1,2, figsize=(20,6))
plt.figure()
axs[0].plot(H.history["loss"], label="train_loss")
axs[0].plot(H.history["val_loss"], label="val_loss")

axs[1].plot(H.history["accuracy"], label="train_acc")
axs[1].plot(H.history["val_accuracy"], label="val_acc")

C:\Users\Mateo\anaconda3\Lib\site-packages\keras\src\applications\mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


10734624/10734624 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Found 12599 images belonging to 16 classes.
Found 3141 images belonging to 16 classes.
Epoch 1/1000


C:\Users\Mateo\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


197/197 ━━━━━━━━━━━━━━━━━━━━ 43s 179ms/step - accuracy: 0.1010 - loss: 2.8065 - val_accuracy: 0.2344 - val_loss: 2.7357
Epoch 2/1000
197/197 ━━━━━━━━━━━━━━━━━━━━ 37s 183ms/step - accuracy: 0.2247 - loss: 2.4783 - val_accuracy: 0.3311 - val_loss: 2.6059
Epoch 3/1000
197/197 ━━━━━━━━━━━━━━━━━━━━ 39s 193ms/step - accuracy: 0.3055 - loss: 2.2870 - val_accuracy: 0.3877 - val_loss: 2.3187
Epoch 4/1000
197/197 ━━━━━━━━━━━━━━━━━━━━ 39s 195ms/step - accuracy: 0.3448 - loss: 2.1619 - val_accuracy: 0.3623 - val_loss: 2.0797
Epoch 5/1000


C:\Users\Mateo\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


197/197 ━━━━━━━━━━━━━━━━━━━━ 41s 203ms/step - accuracy: 0.3583 - loss: 2.0863 - val_accuracy: 0.4023 - val_loss: 1.9892
Epoch 6/1000
197/197 ━━━━━━━━━━━━━━━━━━━━ 40s 201ms/step - accuracy: 0.3833 - loss: 1.9995 - val_accuracy: 0.4414 - val_loss: 1.8530
Epoch 7/1000
197/197 ━━━━━━━━━━━━━━━━━━━━ 36s 179ms/step - accuracy: 0.4096 - loss: 1.9179 - val_accuracy: 0.4541 - val_loss: 1.8546
Epoch 8/1000
197/197 ━━━━━━━━━━━━━━━━━━━━ 39s 191ms/step - accuracy: 0.4189 - loss: 1.8842 - val_accuracy: 0.3768 - val_loss: 1.8778
Epoch 9/1000
197/197 ━━━━━━━━━━━━━━━━━━━━ 39s 194ms/step - accuracy: 0.4478 - loss: 1.8155 - val_accuracy: 0.4785 - val_loss: 1.7253
Epoch 10/1000
197/197 ━━━━━━━━━━━━━━━━━━━━ 40s 198ms/step - accuracy: 0.4510 - loss: 1.8050 - val_accuracy: 0.5098 - val_loss: 1.6953
Epoch 11/1000
113/197 ━━━━━━━━━━━━━━━━━━━━ 16s 194ms/step - accuracy: 0.4598 - loss: 1.7508

KeyboardInterrupt: 

In [2]:
model.evaluate(train_iter)
model.evaluate(valid_iter)

197/197 ━━━━━━━━━━━━━━━━━━━━ 36s 183ms/step - accuracy: 0.5001 - loss: 1.6570
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 198ms/step - accuracy: 0.4792 - loss: 1.7034


[1.7050138711929321, 0.485832542181015]